In [9]:
import numpy as np
import os
import librosa
import soundfile as sf

In [ ]:
MIN_AUDIO_TIMESIZE = 5
SIZE_BY_SECOND = 32000

def segmentar(audio,y,sr):  # input an audio (y-array) to segment
    lista_segmentos=[]
    if len(y) > MIN_AUDIO_TIMESIZE * SIZE_BY_SECOND: # Minimum audio size 5 seconds
        print('Audio a segmentar----->',audio)
        for i in range(0, len(y)-int(MIN_AUDIO_TIMESIZE*SIZE_BY_SECOND),1*sr):
            segment = y[i:i+int(MIN_AUDIO_TIMESIZE*sr)]
            clip = segment.copy()
            lista_segmentos.append(clip)
    return lista_segmentos

In [ ]:
def get_min_energy(L): # enter a list of segments (arrays) of a given audio
    energy_array= np.array([])
    for segment in L:
        rms_segment=np.sqrt(np.sum(segment**2)/len(segment))
        energy_array= np.append(energy_array,rms_segment)
    return np.min(energy_array),np.argmin(energy_array)

In [ ]:
def proccess_sounds(relative_path):
    with open('features_min.csv', 'w') as csv_file:
        line=1
        csv_file.write("line,folder,sub_folder,audio,energy,index\n")
        for species in os.listdir(relative_path):
            for audio in os.listdir(relative_path+"/"+species):
                filename = relative_path+"/"+species+'/'+audio
                y, sr = librosa.load(filename, mono = True, sr=SIZE_BY_SECOND)
                y = y/np.max(np.abs(y))
                L=segmentar(audio,y,sr) 
                if L!=[]:  
                    min_energy,index = get_min_energy(L)
                    csv_file.write(str(line)+","+relative_path+","+species+","+audio+","+str(min_energy)+","+str(index)+"\n")
                    line=line+1
    print("Minima energia dos min filtrado",min_energy) # to take as a reference value as a minimum

In [ ]:
def get_samples(count,seed):
    data = np.genfromtxt('features_min.csv', delimiter=',', dtype=None, names=True, encoding=None) 
    mean=np.mean(data['energy'])
    datos_filtrados = data[data['energy'] < mean]
    print("Dados Filtrados:",len(datos_filtrados))
    print("Mean:",mean)
    np.random.seed(seed)
    np.random.shuffle(datos_filtrados)

    samples = datos_filtrados[:count]

    return samples

In [14]:
def from_sample_to_wav(samples,output_root):
    if not os.path.exists(output_root):
        os.makedirs(output_root)
    for tupla in samples:
        path = tupla['folder']+"/"+tupla['sub_folder']+"/"+tupla['audio']

        y, sr = librosa.load(path, mono = True, sr=32000)
        y = y/np.max(np.abs(y))

        segment = y[tupla['index']*sr:(tupla['index']+MIN_AUDIO_TIMESIZE)*sr]
        
        new_path = output_root+"/silent_"+tupla['sub_folder']+"_"+tupla['audio']+".wav"
        new_path = new_path.replace(".ogg","")

        sf.write(new_path, segment, sr, subtype='PCM_24')

In [ ]:
proccess_sounds("train_audio")
samples = get_samples(4000,42)
from_sample_to_wav(samples,"silence")

Audio a segmentar-----> XC793609.ogg
Audio a segmentar-----> XC124420.ogg
Audio a segmentar-----> XC636663.ogg
Audio a segmentar-----> XC640480.ogg
Audio a segmentar-----> XC615776.ogg
Audio a segmentar-----> XC335666.ogg
Audio a segmentar-----> XC138655.ogg
Audio a segmentar-----> XC639943.ogg
Audio a segmentar-----> XC635412.ogg
Audio a segmentar-----> XC637334.ogg
Audio a segmentar-----> XC637337.ogg
Audio a segmentar-----> XC346561.ogg
Audio a segmentar-----> XC448059.ogg
Audio a segmentar-----> XC747554.ogg
Audio a segmentar-----> XC213965.ogg
Audio a segmentar-----> XC349693.ogg
Audio a segmentar-----> XC615777.ogg
Audio a segmentar-----> XC646710.ogg
Audio a segmentar-----> XC509720.ogg
Audio a segmentar-----> XC615774.ogg
Audio a segmentar-----> XC19895.ogg
Audio a segmentar-----> XC196950.ogg
Audio a segmentar-----> XC164018.ogg
Audio a segmentar-----> XC295481.ogg
Audio a segmentar-----> XC209003.ogg
Audio a segmentar-----> XC633433.ogg
Audio a segmentar-----> XC575154.ogg
Au